Installing the Spark Dependancies

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark==3.0.3

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

Copy the Dataset into the local store (stored under "/content" folder in Colab)

Dataset description can be found here: https://github.com/nytimes/covid-19-data

In [ ]:
! wget https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv

**Start your program**

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession

Create a Spark Session

In [ ]:
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

Read the dataset as a Dataframe

In [ ]:
# Load the dataset
data = spark.read.load('/content/us-counties.csv', format='csv', inferSchema=True, header=True)

# Print schema
data.printSchema()

In [ ]:
# See first 10 rows of the dataset
data.head(10)

**Task 0. Find the daily new cases across the entire US and plot**
(*you DO not need to do this, this code is given for your assistance*)

In [ ]:
from pyspark.sql import functions as F

# Aggregate by day, sum the cases for all counties for each day
daily_cumulative = data.groupby('date').agg(F.sum('cases').alias('total_cases'))
daily_cumulative = daily_cumulative.sort('date')
daily_cumulative.show(10)

# Convert Spark dataframe to a Panda dataframe to plot
plot_data = daily_cumulative.toPandas()
dates = plot_data['date']
values = plot_data['total_cases']

# Find daily news cases from cumulative cases
daily_cases = [values[i+1] - values[i] for i in range(len(values)-1)]
ddates = [dates[i+1] for i in range(len(values)-1)]

import matplotlib.pyplot as plt
import numpy as np
plt.plot(ddates, daily_cases)
plt.xlabel('Date')
plt.ylabel('Daily new cases')
plt.xticks(np.arange(1, len(daily_cases),30), [ddates[i] for i in np.arange(1, len(daily_cases),30)], rotation=90)
plt.show()

Now, solve any **three** tasks from the follwing (BONUS score if you do all!).

*Feel free to add more code blocks withinn each task to spereate the codes for better clarity and understanding.

**Task 1: Find the total number of new cases added in the entire US in the  month of March 2020.**

**Task 2: Calculate the total new cases added in three consecutive months of June, July, and August of 2020 in Jackson county, Missouri (fips code 29095).**

**Task 3: Find the daily new cases per month per 1000 population in Missouri state (MO) since the beginning of the pandemic (assume MO's population is 6,154,913). [Plot the data]**

**Task 4:  On which date all 50 US states have at least 100 cases? At least one death?**

**Task 5: Which single day in the year 2020 had the largest number of deaths in the entire US (if there are multiple such dates, choose the earliest one)?**

Your Warm-up Project ends here.
Thank you.